In [ ]:
%pip install -qU langchain langchain-community langchain-openai python-dotenv numexpr

In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import AIMessage
from langchain_core.runnables import Runnable

In [3]:
load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]
model_name = "gpt-4o"
llm = ChatOpenAI(
    model = model_name,
    openai_api_key = openai_api_key,
    temperature = 1,
    max_tokens = 1000
)

In [4]:
@tool
def count_emails(last_number_of_days: int) -> int:
    """
    This tool is useful to counting emails from the mailbox, based on the last or previous number of days
    """

    return last_number_of_days * 2


@tool
def send_email(email_message: str, recipient: str) -> str:
    """
    This tool is useful when you need to send an email to a recipient
    """

    return f"Successfully Sent the email to {recipient}"


In [22]:
tools = [count_emails, send_email]
llm_with_tools = llm.bind_tools(tools)

In [23]:
import json


def human_approval(message: AIMessage) -> Runnable:
    tool_strings = "\n\n".join(
        json.dumps(tool_call, indent=2) for tool_call in message.tool_calls
    )

    input_message = f"""
        Do you want to approve the following tool invocation {tool_strings} \n\n

        Please enter yes / Y, anything else shall reject the execution of those tools
    """

    response = input(input_message)

    if (response.lower() not in ("yes", "y")):
        raise ValueError("Tool Invocation Not Approved!")

    return message


def call_tools(message: AIMessage) -> Runnable:
    """
    Simple Sequential Tool Calling Helper Function to the LLM
    """
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = message.tool_calls.copy()

    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(
            tool_call["args"])
    
    return tool_calls

In [24]:
chain = llm_with_tools | human_approval | call_tools

In [ ]:
question = "How many emails do i have in my mailbox for the last 10 days? and send me email the report about that, to manager@it-info.com."

response = chain.invoke(question)

print(response)